In [7]:
import requests
from time import sleep
import random
import json
import sqlalchemy
from sqlalchemy import text
from datetime import datetime


class AWSDBConnector:
    def __init__(self):
        self.HOST = "pinterestdbreadonly.cq2e8zno855e.eu-west-1.rds.amazonaws.com"
        self.USER = 'project_user'
        self.PASSWORD = ':t%;yCY3Yjg'
        self.DATABASE = 'pinterest_data'
        self.PORT = 3306

        # Create database connector
        self.engine = self.create_db_connector()

    def create_db_connector(self):
        engine = sqlalchemy.create_engine(
            f"mysql+pymysql://{self.USER}:{self.PASSWORD}@{self.HOST}:{self.PORT}/{self.DATABASE}?charset=utf8mb4")
        return engine

    def query_table(self, table_name: str, row_number: int):
        query_string = text(
            f"SELECT * FROM {table_name} LIMIT {row_number}, 1")
        with self.engine.connect() as connection:
            result = connection.execute(query_string)
            return dict(next(result)._mapping)


class Main:
    invoke_url = 'https://8cijr26k96.execute-api.us-east-1.amazonaws.com/API/topics'

    def __init__(self):
        self.db_connector = AWSDBConnector()

    def run_infinite_post_data_loop(self):
        while True:
            sleep(random.randrange(0, 2))
            random_row = random.randint(0, 11000)
            pin_result = self.db_connector.query_table('pinterest_data', random_row)
            geo_result = self.db_connector.query_table('geolocation_data', random_row)
            user_result = self.db_connector.query_table('user_data', random_row)
            self.to_msk(pin_result, geo_result, user_result)

    @staticmethod
    def post(method: str, url: str, data: dict):
        payload = json.dumps({"records": [{"value": data}]})
        headers = {'Content-Type': 'application/vnd.kafka.json.v2+json'}
        response = requests.request(method, url, headers=headers, data=payload)
        return response

    def to_msk(self, pin_result, geo_result, user_result) -> None:
        # Convert datetime objects to ISO format strings
        if 'timestamp' in geo_result:
            geo_result['timestamp'] = geo_result['timestamp'].isoformat()
        if 'date_joined' in user_result:
            user_result['date_joined'] = user_result['date_joined'].isoformat()

        responses = {
            "pin_response": self.post("POST", f"{self.invoke_url}/0e3af383e4cd.pin", pin_result),
            "geo_response": self.post("POST", f"{self.invoke_url}/0e3af383e4cd.geo", geo_result),
            "user_response": self.post("POST", f"{self.invoke_url}/0e3af383e4cd.user", user_result),
        }

        for key, response in responses.items():
            print(f"Response content: {response.content.decode('utf-8')}\n")
            print(f"{key}: {response.status_code}\n")
            


# Example usage:
if __name__ == "__main__":
    main_instance = Main()
    main_instance.run_infinite_post_data_loop()

Response content: {"message": "Network error communicating with endpoint"}

pin_response: 504

Response content: {"message": "Network error communicating with endpoint"}

geo_response: 504

Response content: {"message": "Network error communicating with endpoint"}

user_response: 504



KeyboardInterrupt: 

In [8]:
import requests
from time import sleep
import random
import boto3
import json
import sqlalchemy
from sqlalchemy import text


random.seed(100)


class Main:
    invoke_url = 'https://8cijr26k96.execute-api.us-east-1.amazonaws.com/API'

    def __init__(self):

        self.HOST = "pinterestdbreadonly.cq2e8zno855e.eu-west-1.rds.amazonaws.com"
        self.USER = 'project_user'
        self.PASSWORD = ':t%;yCY3Yjg'
        self.DATABASE = 'pinterest_data'
        self.PORT = 3306

        # Create database connector
        self.engine = self.create_db_connector()

    def create_db_connector(self):
        engine = sqlalchemy.create_engine(
            f"mysql+pymysql://{self.USER}:{self.PASSWORD}@{self.HOST}:{self.PORT}/{self.DATABASE}?charset=utf8mb4")
        return engine

    def query_table(self, table_name: str, row_number: int):
        query_string = text(
            f"SELECT * FROM {table_name} LIMIT {row_number}, 1")
        with self.engine.connect() as connection:
            result = connection.execute(query_string)
            return dict(next(result)._mapping)

    def run_emulation_cycle(self):
        sleep(random.randrange(0, 2))
        random_row = random.randint(0, 11000)
        return {"pin": self.query_table('pinterest_data', random_row),
                "geo": self.query_table('geolocation_data', random_row),
                "user": self.query_table('user_data', random_row)}

    @staticmethod
    def post(method: str, url: str, data: dict):
        payload = json.dumps({"records": [{"value": data}]})
        headers = {'Content-Type': 'application/vnd.kafka.json.v2+json'}
        response = requests.request(method, url, headers=headers, data=payload)
        return response

    def to_console(self, pin_result: dict, geo_result: dict, user_result: dict) -> None:
        print(f"PIN:\n{pin_result}\n")
        print(f"GEO:\n{geo_result}\n")
        print(f"USER:\n{user_result}\n")

    def to_msk(self, pin_result, geo_result, user_result) -> None:
        responses = {
            "pin_response": self.post("POST", f"{self.invoke_url}/topics/0e3af383e4cd.pin", pin_result),
            "geo_response": self.post("POST", f"{self.invoke_url}/topics/0e3af383e4cd.geo", geo_result),
            "user_response": self.post("POST", f"{self.invoke_url}/topics/0e3af383e4cd.user", user_result),
        }

        for key, response in responses.items():
            print(f"{key}: {response.status_code}\n")


# Example usage:
if __name__ == "__main__":
    main_instance = Main()
    data = main_instance.run_emulation_cycle()
    main_instance.to_console(data["pin"], data["geo"], data["user"])
    main_instance.to_msk(data["pin"], data["geo"], data["user"])

PIN:
{'index': 7528, 'unique_id': 'fbe53c66-3442-4773-b19e-d3ec6f54dddf', 'title': 'No Title Data Available', 'description': 'No description available Story format', 'poster_name': 'User Info Error', 'follower_count': 'User Info Error', 'tag_list': 'N,o, ,T,a,g,s, ,A,v,a,i,l,a,b,l,e', 'is_image_or_video': 'multi-video(story page format)', 'image_src': 'Image src error.', 'downloaded': 0, 'save_location': 'Local save in /data/mens-fashion', 'category': 'mens-fashion'}

GEO:
{'ind': 7528, 'timestamp': datetime.datetime(2020, 8, 28, 3, 52, 47), 'latitude': -89.9787, 'longitude': -173.293, 'country': 'Albania'}

USER:
{'ind': 7528, 'first_name': 'Abigail', 'last_name': 'Ali', 'age': 20, 'date_joined': datetime.datetime(2015, 10, 24, 11, 23, 51)}



TypeError: Object of type datetime is not JSON serializable

In [9]:
import requests
from time import sleep
import random
from multiprocessing import Process
import boto3
import json
import sqlalchemy
from sqlalchemy import text


random.seed(100)


class AWSDBConnector:

    def __init__(self):

        self.HOST = "pinterestdbreadonly.cq2e8zno855e.eu-west-1.rds.amazonaws.com"
        self.USER = 'project_user'
        self.PASSWORD = ':t%;yCY3Yjg' 
        self.DATABASE = 'pinterest_data'
        self.PORT = 3306
        
    def create_db_connector(self):
        engine = sqlalchemy.create_engine(f"mysql+pymysql://{self.USER}:{self.PASSWORD}@{self.HOST}:{self.PORT}/{self.DATABASE}?charset=utf8mb4")
        return engine


new_connector = AWSDBConnector()


def run_infinite_post_data_loop():
    while True:
        sleep(random.randrange(0, 2))
        random_row = random.randint(0, 11000)
        engine = new_connector.create_db_connector()
        invoke_url = "https://8cijr26k96.execute-api.us-east-1.amazonaws.com/API/topic/0e3af383e4cd.pin"

        with engine.connect() as connection:

            pin_string = text(f"SELECT * FROM pinterest_data LIMIT {random_row}, 1")
            pin_selected_row = connection.execute(pin_string)
            
            for row in pin_selected_row:
                pin_result = dict(row._mapping)
                payload = json.dumps({
                    "records": [
                        {
                            
                        "value": {"index": pin_result["index"], "unique_id": pin_result["unique_id"], "title": pin_result["title"], "description": pin_result["description"], "poster_name": pin_result["poster_name"], "follower_count": pin_result["follower_count"], "tag_list": pin_result["tag_list"], "is_image_or_video": pin_result["is_image_or_video"], "image_src": pin_result["image_src"], "downloaded": pin_result["downloaded"], "save_location": pin_result["save_location"], "category": pin_result["category"]}
                        }
                    ]
                })
                headers = {'Content-Type': 'application/vnd.kafka.json.v2+json'}
                response = requests.request("POST", invoke_url, headers=headers, data=payload)
                print(response.text)

            # geo_string = text(f"SELECT * FROM geolocation_data LIMIT {random_row}, 1")
            # geo_selected_row = connection.execute(geo_string)
            
            # for row in geo_selected_row:
            #     geo_result = dict(row._mapping)

            # user_string = text(f"SELECT * FROM user_data LIMIT {random_row}, 1")
            # user_selected_row = connection.execute(user_string)
            
            # for row in user_selected_row:
            #     user_result = dict(row._mapping)
            
            # print(pin_result)
            # print(geo_result)
            # print(user_result)


if __name__ == "__main__":
    run_infinite_post_data_loop()
    print('Working')

{"message":"Missing Authentication Token"}


KeyboardInterrupt: 